In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x3960b90
Welcome to JupyROOT 6.26/10


In [2]:
ROOT.EnableImplicitMT(6)

In [3]:
%jsroot on

In [4]:
ROOT.gSystem.Load("libedm4hep")
ROOT.gSystem.Load("libpodio")
ROOT.gSystem.Load("libFCCAnalyses")

0

In [5]:
_edm  = ROOT.edm4hep.ReconstructedParticleData()
_pod  = ROOT.podio.ObjectID()
_fcc  = ROOT.dummyLoader

In [6]:
ROOT.TH1.SetDefaultSumw2()

In [7]:
lumi = {
    "eL.pL": 1049,
    "eL.pR": 5005,
    "eR.pL": 5767,
    "eR.pR": 1049,
}

events = {
    "eL.pL": 200000,
    "eL.pR": 51371800,
    "eR.pL": 500000,
    "eR.pR": 200000,
}

evts_per_fb = {}
for pol in lumi:
    evts_per_fb[pol] = events[pol] / lumi[pol]
    
weights = {}
for pol in evts_per_fb:
    weights[pol] = evts_per_fb[pol] / max(evts_per_fb.values())

In [8]:
print(evts_per_fb)
print(weights)

{'eL.pL': 190.65776930409913, 'eL.pR': 10264.095904095904, 'eR.pL': 86.70019074041963, 'eR.pR': 190.65776930409913}
{'eL.pL': 0.01857521315910706, 'eL.pR': 1.0, 'eR.pL': 0.008446938878057616, 'eR.pR': 0.01857521315910706}


In [9]:
files = {
    "eL.pL": "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pL.I500105.0_edm4hep.root",
    "eL.pR": "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0_edm4hep.root",
    "eR.pL": "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eR.pL.I500108.0_edm4hep.root",
    "eR.pR": "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eR.pR.I500107.0_edm4hep.root",
}
#df = ROOT.RDataFrame("events", "test_4f_sw_sl_eL_pR_edm4hep.root")
#df = ROOT.RDataFrame("events", "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0_edm4hep.root")
#df = ROOT.RDataFrame("events", "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eR.pL.I500108.0_edm4hep.root")
#df = ROOT.RDataFrame("events", files)

df = {}
for pol in files:
    df[pol] = ROOT.RDataFrame("events", files[pol])

In [10]:
df["eL.pR"].Describe()

Dataframe from TChain events in file E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0_edm4hep.root

Property                Value
--------                -----
Columns in total           39
Columns from defines        0
Event loops run             0
Processing slots            6

Column                                  Type                                            Origin
------                                  ----                                            ------
EventHeader                             ROOT::VecOps::RVec<edm4hep::EventHeaderData>    Dataset
EventHeader.eventNumber                 ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.runNumber                   ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.timeStamp                   ROOT::VecOps::RVec<ULong_t>                     Dataset
EventHeader.weight                      ROOT::VecOps::RVec<Float_t>                     Dataset
MCParticles                             ROOT::V

In [11]:
%%cpp
ROOT::VecOps::RVec<edm4hep::MCParticleData> get_enu(ROOT::VecOps::RVec<edm4hep::MCParticleData> in) {
    ROOT::VecOps::RVec<edm4hep::MCParticleData> res;
    for (size_t i = 0; i <= in.size(); i++) {
        const auto& mc = in[i];
        const auto& mc_next = in[i+1];
        if (abs(mc.PDG) == 11 && abs(mc_next.PDG) == 12) {
            res.push_back(mc);
            res.push_back(mc_next);
            break;
        }
    }
    return res;
}


In [12]:
%%cpp
ROOT::VecOps::RVec<edm4hep::MCParticleData> get_qq(ROOT::VecOps::RVec<edm4hep::MCParticleData> in) {
    ROOT::VecOps::RVec<edm4hep::MCParticleData> res;
    for (size_t i = 0; i <= in.size(); i++) {
        const auto& mc = in[i];
        const auto& mc_next = in[i+1];
        if (abs(mc.PDG) <= 5 && abs(mc_next.PDG) <= 5) {
            res.push_back(mc);
            res.push_back(mc_next);
            break;
        }
    }
    return res;
}

In [13]:
def defines(df):
       df2 = (df
              .Define("enu", "get_enu(MCParticles)")
              .Define("enu_p", "FCCAnalyses::MCParticle::get_p(enu)")
              .Define("enu_theta", "FCCAnalyses::MCParticle::get_theta(enu)")
              .Define("e", "enu[0]")
              .Define("nu", "enu[1]")
              .Define("e_p", "enu_p[0]")
              .Define("nu_p", "enu_p[1]")
              .Define("e_theta", "enu_theta[0]")
              .Define("nu_theta", "enu_theta[1]")
              .Define("e_seen", "abs(cos(e_theta)) < 0.989") # no silicon hits in ild below 8.5 deg
              .Define("enu_tlv", "FCCAnalyses::MCParticle::get_tlv(enu)")
              .Define("enu_W_tlv", "enu_tlv[0] + enu_tlv[1]")
              .Define("enu_invM", "(enu_tlv[0] + enu_tlv[1]).M()")
              .Define("qq", "get_qq(MCParticles)")
              .Define("qq_tlv", "FCCAnalyses::MCParticle::get_tlv(qq)")
              .Define("qq_W_tlv", "qq_tlv[0] + qq_tlv[1]")
              .Define("qq_invM", "qq_W_tlv.M()")
              .Define("qq_W_theta", "qq_W_tlv.Theta()")
              .Define("cos_theta_W", "cos(qq_W_theta)")
              .Define("e_tlv", "enu_tlv[0]")
              .Define("e_lv", "ROOT::Math::PxPyPzEVector(e_tlv.Px(), e_tlv.Py(), e_tlv.Pz(), e_tlv.E())")
              .Define("enu_lv", "ROOT::Math::PxPyPzEVector(enu_W_tlv.Px(), enu_W_tlv.Py(), enu_W_tlv.Pz(), enu_W_tlv.E())")
              .Define("qq_W_lv", "ROOT::Math::PxPyPzEVector(qq_W_tlv.Px(), qq_W_tlv.Py(), qq_W_tlv.Pz(), qq_W_tlv.E())")
              .Define("WW_lv", "qq_W_lv + enu_lv")
              .Define("e_enu_cms_lv", "ROOT::Math::Boost(enu_lv.BoostToCM())(e_lv)") # boost e to cms of enu
              .Define("decay_theta", "ROOT::Math::VectorUtil::Angle(e_enu_cms_lv.Vect(), enu_lv.Vect())")
              #.Define("decay_phi", "ROOT::Math::VectorUtil::Angle(e_enu_cms_lv.Vect(), ROOT::Math::XYZVector(0, 0, e.charge).Cross(enu_lv.Vect().Unit()))")
              #.Define("decay_phi", "ROOT::Math::VectorUtil::Angle(e_enu_cms_lv.Vect(), ROOT::Math::XYZVector(0, 0, e.charge))")
              .Define("decay_phi", "ROOT::Math::VectorUtil::Angle(e_enu_cms_lv.Vect(), ROOT::Math::XYZVector(0, 0, 1))") # this is still not right as it takes the shorter angle?
              .Define("decay_cos_theta", "cos(decay_theta)")
              )
       return df2

In [14]:
df2 =  {}
for pol in df:
    df2[pol] = defines(df[pol])

In [27]:
h = {}
for pol in df2:
    h[pol] = {}
    h[pol]["e_p"] = df2[pol].Histo1D(("", ";e Momentum [GeV]", 50, 0, 150), "e_p")
    h[pol]["nu_p"] = df2[pol].Histo1D(("", ";#nu Momentum [GeV]", 50, 0, 150), "nu_p")
    h[pol]["e_theta"] = df2[pol].Histo1D("e_theta")
    h[pol]["nu_theta"] = df2[pol].Histo1D("nu_theta")
    #h[pol]["enu_invM"] = df2[pol].Histo1D("enu_invM")
    #h[pol]["qq_invM"] = df2[pol].Histo1D("qq_invM")
    h[pol]["cos_theta_Wp"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} cos #theta_{W}", 50, -1, 1), "cos_theta_W")
    h[pol]["cos_theta_Wm"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} cos #theta_{W}", 50, -1, 1), "cos_theta_W")
    h[pol]["cos_theta_W"] = df2[pol].Histo1D(("", ";cos #theta_{W}", 50, -1, 1), "cos_theta_W")
    h[pol]["decay_cos_theta_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} cos #theta_{l}", 50, -1, 1), "decay_cos_theta")
    h[pol]["decay_cos_theta_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} cos #theta_{l}", 50, -1, 1), "decay_cos_theta")
    h[pol]["decay_cos_theta"] = df2[pol].Histo1D(("", ";cos #theta_{l}", 50, -1, 1), "decay_cos_theta")
    h[pol]["decay_phi_em"] = df2[pol].Filter("e.charge == -1").Histo1D(("", ";e^{-} #phi_{l} [rad]", 50, 0, 3.142), "decay_phi")
    h[pol]["decay_phi_ep"] = df2[pol].Filter("e.charge == +1").Histo1D(("", ";e^{+} #phi_{l} [rad]", 50, 0, 3.142), "decay_phi")
    h[pol]["decay_phi"] = df2[pol].Histo1D(("", ";#phi_{l} [rad]", 50, 0, 3.142), "decay_phi")

In [28]:
#h["enu_invM_log"] = h["enu_invM"].Clone()
h_sum = {}
first_pol = list(h)[0]
for v in h[first_pol]:
    h_sum[v] = h[first_pol][v].Clone()
    h_sum[v].Scale(weights[first_pol])
for pol in list(h)[1:]:
    for v in h[pol]:
        tmp = h[pol][v].Clone()
        #tmp.Scale(weights[pol])
        h_sum[v].Add(tmp, weights[pol])

Info in <TH1D::Add>: Attempt to add histograms with different axis limits - trying to use TH1::Merge
Error in <Merge>: Cannot merge histograms - limits are inconsistent:
 first: e_theta (128, 0.018566, 6.259066), second: e_theta (128, 0.021113, 6.223641)
Info in <TH1D::Add>: Attempt to add histograms with different axis limits - trying to use TH1::Merge
Error in <Merge>: Cannot merge histograms - limits are inconsistent:
 first: nu_theta (128, 0.004122, 6.253769), second: nu_theta (128, 0.006187, 6.263218)
Warning in <TH1D::Add>: Attempt to add histograms with different axis limits
Warning in <TH1D::Add>: Attempt to add histograms with different axis limits
Warning in <TH1D::Add>: Attempt to add histograms with different axis limits
Warning in <TH1D::Add>: Attempt to add histograms with different axis limits


In [29]:
c = {}
for v in h_sum:
    c[v] = ROOT.TCanvas()
    h_sum[v].Draw("e1")
    if (v.endswith("_log")):
        c[v].SetLogy()
    c[v].Draw()
    #c[v].SaveAs(f"plots/{v}.png")
    

